In [ ]:
# @title Instruções {"run":"auto","display-mode":"form"}
from IPython.display import HTML, display

display(HTML('''
<div style="padding: 15px; border-radius: 8px; position: relative; height: 100px">
  <div style="display: flex; flex-direction: column; max-width: 600px; background: #1e1e1e; border: #616161; border-radius: 2px; padding: 12px 8px; white-space: pre; padding: 5px 10px; position: absolute; left: 0; right: 0; margin: 0 auto; ">
    <h2 style="color: white; font-weight: bold; margin-top: 0;">⚠️ Atenção</h2>
    <p style="font-size: 16px;">
      Para executar todo o notebook, pressione: <strong>Ctrl + F9</strong><br>
      ou clique no menu <b>Ambiente de execução → Executar tudo</b>
    </p>
  </div>
</div>
'''))

# Instruções
```
# Faça upload do arquivo de agendamentos extraido
# Os objetos do arquivo serão convertidos de HTML para CSV
```

In [1]:
# @title Execução {"display-mode":"form"}
from google.colab import output
import json
import os
from google.colab import files
from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
import warnings
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
import pandas as pd
from IPython.display import display, HTML, Javascript
import re
from google.colab import auth
from googleapiclient.discovery import build

globals()["DEBUG_MODE"] = False
globals()['DEBUG_MODE'] and print(f"⚠️ DEBUG_MODE atual: {globals()['DEBUG_MODE']}")
display(Javascript(f"window.DEBUG_MODE = {str(globals()['DEBUG_MODE']).lower()};"))


def log_selecao_local(nome_arquivo):
    print(f"📁 Arquivo local selecionado: {nome_arquivo}")

output.register_callback('log_selecao_local', log_selecao_local)

def callback_mensagem_conversao():
    print("🔁 Arquivo sendo convertido para CSV...")

output.register_callback("callback_mensagem_conversao", callback_mensagem_conversao)

def callback_diretorio_escolhido(caminho_completo):
    print(f"📂 Diretório escolhido para salvar: {caminho_completo}")

output.register_callback("callback_diretorio_escolhido", callback_diretorio_escolhido)

def callback_print(msg):
    print(f"{msg}")

output.register_callback("callback_print", callback_print)


def strip(txt):
    return txt.replace("\n", "").replace("\r", "").replace("\\n", "").replace("\\r", "").strip()

from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display, HTML

def strip(txt):
    return txt.replace("\n", "").replace("\r", "").replace("\\n", "").replace("\\r", "").strip()

def converter_html_para_csv(html_ou_lista, nome_base="agendamentos"):
    callback_mensagem_conversao()
    if isinstance(html_ou_lista, str):
        try:
            html_list = json.loads(html_ou_lista)
            if not isinstance(html_list, list):
                html_list = [html_ou_lista]
        except:
            html_list = [html_ou_lista]
    elif isinstance(html_ou_lista, list):
        html_list = html_ou_lista
    else:
        raise Exception("❌ Tipo de entrada inválido.")

    dados = []
    for html in html_list:
        soup = BeautifulSoup(html, "html.parser")
        tabelas = soup.find_all("table")
        tabela_certa = None
        for tabela in tabelas:
            if "Data" in tabela.text and "Hora" in tabela.text:
                tabela_certa = tabela
                break

        if not tabela_certa:
            continue

        linhas = tabela_certa.find("tbody").find_all("tr")
        for linha in linhas:
            colunas = linha.find_all("td")
            if len(colunas) >= 11:
                dados.append({
                    "Data": strip(colunas[0].text),
                    "Hora": strip(colunas[1].text),
                    "Profissional": strip(colunas[3].text),
                    "Serviço": strip(colunas[4].text),
                    "Duração": strip(colunas[5].text),
                    "Cliente": strip(colunas[6].text),
                    "Valor": strip(colunas[7].text).replace("R$", "").replace(",", ".").strip(),
                    "Status": strip(colunas[8].text),
                    "Serviço com Foto": strip(colunas[10].text),
                    "Cadastramento": strip(colunas[11].text) if len(colunas) > 11 else ""
                })

    if not dados:
        callback_print("🚫 Nenhuma linha de dados encontrada.")
        raise Exception("🚫 Nenhuma linha de dados foi encontrada.")

    df = pd.DataFrame(dados)
    caminho_csv = f"/content/{nome_base}.csv"
    df.to_csv(caminho_csv, index=False, encoding="utf-8-sig")
    print(f"✅ CSV gerado com sucesso: `{caminho_csv}`")

    html_ui = f"""
    <div style='font-family:sans-serif;'>
        <p><strong>✅ CSV pronto:</strong> <code>{nome_base}.csv</code></p>
        <div id="menu_csv_acao" style='display: flex; gap: 10px; margin-bottom: 12px;'>
            <button onclick="
                document.getElementById('menu_csv_acao')?.remove();
                google.colab.kernel.invokeFunction('baixar_csv', ['{caminho_csv}'], {{}});
            " style='background-color:#3b82f6;color:white;padding:8px 16px;border:none;border-radius:4px;cursor:pointer;'>
                📥 Baixar CSV
            </button>
            <button onclick="
              document.getElementById('menu_csv_acao')?.remove();
              initDriveUI('{caminho_csv}', {{
                modo: 'salvar',
                tipos_permitidos: ['.csv'],
                mostrar_arquivos: false,
                mostrar_botao_renomear: true,
                mostrar_botao_excluir: true,
                mostrar_botoes_acao: true
              }});
            "
                    style='background-color:#10b981;color:white;padding:8px 16px;border:none;border-radius:4px;cursor:pointer;'>
                ☁️ Salvar no Google Drive
            </button>
        </div>
        <div id='input_drive'></div>
    </div>
    <script>
    window.initDriveUI = function(caminho, config) {{
      const existente = document.getElementById('input_drive');
      if (existente) existente.remove();
      const div = document.createElement('div');
      div.id = 'input_drive';
      div.innerHTML = '<p>📁 Carregando pastas...</p>';
      document.body.appendChild(div);
      const jsonConfig = JSON.stringify(config || {{}});
      google.colab.kernel.invokeFunction('listar_pastas_drive_js', [caminho, '', '[]', jsonConfig], {{}});
    }};
    </script>
    """
    display(HTML(html_ui))

def baixar_csv(caminho):
    print("📦 Iniciando download...")
    files.download(caminho)
    print("✅ Download iniciado!")

output.register_callback('baixar_csv', baixar_csv)

# JavaScript com UI, toast, bloqueio e botões dinâmicos
display(Javascript('''
// Aqui está o início do código com logs de debug adicionados
// window.initDriveUI = function(caminho, config) {
//   console.log("🔍 initDriveUI chamado", caminho, config);
//   var div = document.getElementById('input_drive');
//   div.innerHTML = '<p>📁 Carregando pastas...</p>';
//   const jsonConfig = JSON.stringify(config || {});
//   console.log("🧠 Chamando listar_pastas_drive_js com:", caminho, jsonConfig);
//   google.colab.kernel.invokeFunction('listar_pastas_drive_js', [caminho, '', '[]', jsonConfig], {});
// };
window.initDriveUI = function(caminho, config) {
  DEBUG_MODE && console.log("🔍 initDriveUI chamado", caminho, config);

  // 🔁 Se já existe um #input_drive, remove ele
  const existente = document.getElementById('input_drive');
  if (existente) {
    existente.remove();
  }

  // ✅ Cria um novo container limpo
  const div = document.createElement('div');
  div.id = 'input_drive';
  div.innerHTML = '<p>📁 Carregando pastas...</p>';
  document.body.appendChild(div);

  // 🔄 Chama o Python para listar pastas
  const jsonConfig = JSON.stringify(config || {});
  google.colab.kernel.invokeFunction('listar_pastas_drive_js', [caminho, '', '[]', jsonConfig], {});
};



window.renderFolderUI = function(html) {
  DEBUG_MODE && console.log("📦 renderFolderUI executado");

  const container = document.getElementById('input_drive');
  const temp = document.createElement('div');
  temp.innerHTML = html;

  const rootDiv = temp.querySelector("#drive_ui_container");
  const isAppend = rootDiv?.dataset?.append === "true";
  DEBUG_MODE && console.log("📦 UI render", { append: isAppend, rootDiv });

  if (!isAppend) {
    container.innerHTML = html;
    return;
  }

  const novaUL = temp.querySelector("ul");
  const ulExistente = container.querySelector("ul");
  if (novaUL && ulExistente) {
    novaUL.querySelectorAll("li").forEach(li => ulExistente.appendChild(li));
    const novoBotaoMais = temp.querySelector("#carregar_mais_btn");
    const botaoAntigo = container.querySelector("#carregar_mais_btn");
    if (botaoAntigo) {
      if (novoBotaoMais) botaoAntigo.replaceWith(novoBotaoMais);
      else botaoAntigo.remove();
    } else if (novoBotaoMais) {
      ulExistente.insertAdjacentElement("afterend", novoBotaoMais);
    }
  } else {
    container.innerHTML = html;
  }
};

window.selectFile = function(fileId, fileName, btn) {
  DEBUG_MODE && console.log("📂 Arquivo selecionado", fileId, fileName);

  // Desabilita todos os botões da interface
  const allBtns = document.querySelectorAll("#input_drive button");
  allBtns.forEach(b => {
    b.disabled = true;
    b.style.opacity = 0.4;
    b.style.cursor = "not-allowed";
  });

  // Remove a interface visual
  const driveDiv = document.getElementById("input_drive");
  if (driveDiv) driveDiv.remove();

  // Chama o Python para baixar e processar o arquivo
  google.colab.kernel.invokeFunction(
    "carregar_arquivo_selecionado",
    [fileId, fileName],
    {}
  );

  DEBUG_MODE && console.log("⏳ Processando no backend...");
};



window.showToast = function(message, type = "success") {
  DEBUG_MODE && console.log("🔔 Toast mostrado", message, type);
  const toast = document.getElementById('toast_drive_msg');
  toast.innerText = message;
  toast.style.background = type === "error" ? "#dc2626" : "#16a34a";
  toast.style.display = 'block';
  setTimeout(() => { toast.style.display = 'none'; }, 3000);
};

window.disableDriveUIButtons = function(clickedButton, label) {
  DEBUG_MODE && console.log("🚫 Botões desativados", clickedButton, label);
  const buttons = document.querySelectorAll('#input_drive button');
  buttons.forEach(btn => {
    btn.disabled = true;
    btn.style.opacity = 0.4;
    btn.style.cursor = "not-allowed";
    if (btn === clickedButton) {
      btn.dataset.originalText = btn.innerText;
      btn.innerText = `⏳ ${label}...`;
    }
  });
};

window.enableDriveUIButtons = function() {
  DEBUG_MODE && console.log("✅ Botões reativados");
  const buttons = document.querySelectorAll('#input_drive button');
  buttons.forEach(btn => {
    btn.disabled = false;
    if (btn.dataset.originalText) {
      btn.innerText = btn.dataset.originalText;
      delete btn.dataset.originalText;
    }
  });
};

window.disableAllDriveButtons = function(clickedBtn, label = "Carregando") {
  const allBtns = document.querySelectorAll(".file-btn, .breadcrumb-btn");
  allBtns.forEach(btn => {
    btn.disabled = true;
    btn.style.opacity = 0.4;
    btn.style.cursor = "not-allowed";

    // Apenas o botão clicado muda de texto
    if (btn === clickedBtn) {
      if (!btn.dataset.originalText) {
        btn.dataset.originalText = btn.innerText;
      }
      btn.innerText = `⏳ ${label}...`;
    }
  });
};

function promptNewFolderName(caminho, pathStackJson) {
  const name = prompt("Nome da nova pasta:");
  if (name) {
    const btn = event.target;
    disableDriveUIButtons(btn, "Criando");
    google.colab.kernel.invokeFunction("criar_pasta", [name, caminho, pathStackJson], {})
      .then(() => enableDriveUIButtons());
  }
}

function promptRenameFolder(folderId, currentName, caminho, pathStackJson) {
  const name = prompt("Renomear pasta:", currentName);
  if (name && name !== currentName) {
    const btn = event.target;
    disableDriveUIButtons(btn, "Renomeando");
    google.colab.kernel.invokeFunction("renomear_pasta", [folderId, name, caminho, pathStackJson], {})
      .then(() => enableDriveUIButtons());
  }
}

function confirmDeleteFolder(folderId, caminho, pathStackJson) {
  if (confirm("Tem certeza que deseja excluir esta pasta?")) {
    const btn = event.target;
    disableDriveUIButtons(btn, "Excluindo");
    google.colab.kernel.invokeFunction("excluir_pasta", [folderId, caminho, pathStackJson], {})
      .then(() => enableDriveUIButtons());
  }
}
'''))

# Tradução de erros comuns
def traduzir_erro(e):
    mensagem = str(e).lower()
    if "already exists" in mensagem:
        return ("🗂 Já existe uma pasta com esse nome", "error")
    elif "permission" in mensagem:
        return ("❗ Você não tem permissão para essa ação", "error")
    elif "not found" in mensagem:
        return ("🔍 Pasta não encontrada", "error")
    else:
        return (f"🚫 Erro: {str(e).splitlines()[0]}", "error")

# Ações com toast
def criar_pasta(nome, caminho, path_stack_json):
    try:
        auth.authenticate_user()
        drive_service = build('drive', 'v3')
        parent_id = json.loads(path_stack_json)[-1]['id'] if json.loads(path_stack_json) else 'root'
        drive_service.files().create(body={
            'name': nome,
            'mimeType': 'application/vnd.google-apps.folder',
            'parents': [parent_id]
        }, fields='id').execute()
        display(Javascript("showToast('📁 Pasta criada com sucesso!', 'success'); enableDriveUIButtons();"))
        listar_pastas_drive_js(caminho, '', path_stack_json, '')
    except Exception as e:
        msg, tipo = traduzir_erro(e)
        msg = msg.replace("'", "\\'")
        display(Javascript(f"showToast('{msg}', '{tipo}'); enableDriveUIButtons();"))

def renomear_pasta(folder_id, novo_nome, caminho, path_stack_json):
    try:
        auth.authenticate_user()
        drive_service = build('drive', 'v3')
        drive_service.files().update(fileId=folder_id, body={'name': novo_nome}).execute()
        display(Javascript("showToast('✏️ Pasta renomeada com sucesso!', 'success'); enableDriveUIButtons();"))
        listar_pastas_drive_js(caminho, '', path_stack_json, '')
    except Exception as e:
        msg, tipo = traduzir_erro(e)
        msg = msg.replace("'", "\\'")
        display(Javascript(f"showToast('{msg}', '{tipo}'); enableDriveUIButtons();"))

def excluir_pasta(folder_id, caminho, path_stack_json):
    try:
        auth.authenticate_user()
        drive_service = build('drive', 'v3')
        drive_service.files().delete(fileId=folder_id).execute()
        path_stack = json.loads(path_stack_json)
        if path_stack and path_stack[-1]['id'] == folder_id:
            path_stack = path_stack[:-1]
        display(Javascript("showToast('❌ Pasta excluída com sucesso!', 'success'); enableDriveUIButtons();"))
        listar_pastas_drive_js(caminho, '', json.dumps(path_stack), '')
    except Exception as e:
        msg, tipo = traduzir_erro(e)
        msg = msg.replace("'", "\\'")
        display(Javascript(f"showToast('{msg}', '{tipo}'); enableDriveUIButtons();"))

# Renderização e navegação
def botao_html(texto, onclick, cor_base, cor_hover):
    return f"""
    <button onclick='{onclick}'
            style='background:{cor_base};color:white;padding:8px 14px;border:none;border-radius:6px;
                   font-size:14px;box-shadow:0 1px 3px rgba(0,0,0,0.3);transition:all 0.2s;
                   cursor:pointer;'>
        {texto}
    </button>
    """

ITENS_POR_PAGINA = 25  # 👈 Altere aqui facilmente

def listar_pastas_drive_js(caminho, page_token, path_stack_json, config_json):
    globals()['DEBUG_MODE'] and print(f"🔃 listando pastas e arquivos - caminho: {caminho}, token: {page_token}, path_stack: {path_stack_json}, config: {config_json}")
    # Garante que path_stack_json seja uma string JSON válida
    if isinstance(path_stack_json, list):
        path_stack_json = json.dumps(path_stack_json)
    if isinstance(config_json, dict):
        config_json = json.dumps(config_json)

    path_stack = json.loads(path_stack_json)
    config = json.loads(config_json) if config_json else {}
    parent_id = path_stack[-1]['id'] if path_stack else 'root'

    globals()['DEBUG_MODE'] and print(f"📁 Navegando para parent_id: {parent_id}")

    auth.authenticate_user()
    drive_service = build('drive', 'v3')

    # path_stack = json.loads(path_stack_json)
    # config = json.loads(config_json) if config_json else {}
    # parent_id = path_stack[-1]['id'] if path_stack else 'root'
    query_base = f"'{parent_id}' in parents and trashed=false"

    exibir_arquivos = config.get("mostrar_arquivos", False)
    tipos_permitidos = config.get("tipos_permitidos", [])
    mostrar_botoes_acao = config.get("mostrar_botoes_acao", True)
    modo = config.get("modo", "salvar")

    folders = []
    arquivos = []
    folder_token = ''
    file_token = ''

    # 🔹 DEBUG: Tipos permitidos
    globals()['DEBUG_MODE'] and print(f"📦 Tipos permitidos: {tipos_permitidos}")

    # 1️⃣ Carrega pastas se necessário
    if not config.get("pastas_carregadas"):
        token_limpo = None if page_token in ("", "null") else page_token
        globals()['DEBUG_MODE'] and print(f"🛠 token_limpo usado para pastas: {repr(token_limpo)}")

        query_folders = f"{query_base} and mimeType='application/vnd.google-apps.folder'"
        resp_folders = drive_service.files().list(
            q=query_folders,
            spaces='drive',
            fields="nextPageToken, files(id, name, mimeType)",
            pageSize=ITENS_POR_PAGINA,
            pageToken=token_limpo
        ).execute()

        folders = resp_folders.get("files", [])
        folder_token = resp_folders.get("nextPageToken", '')

        globals()['DEBUG_MODE'] and print(f"📁 Pastas carregadas: {len(folders)}")

        # ⚡️ Se poucas pastas, já carrega arquivos
        if len(folders) < ITENS_POR_PAGINA and exibir_arquivos:
            globals()['DEBUG_MODE'] and print("⚡️ Pastas insuficientes, carregando arquivos imediatamente...")
            config["pastas_carregadas"] = True
            query_files = f"{query_base} and mimeType!='application/vnd.google-apps.folder'"
            resp_files = drive_service.files().list(
                q=query_files,
                spaces='drive',
                fields="nextPageToken, files(id, name, mimeType)",
                pageSize=ITENS_POR_PAGINA,
                pageToken=None
            ).execute()

            arquivos = resp_files.get("files", [])
            file_token = resp_files.get("nextPageToken", '')

    # 2️⃣ Carrega arquivos se já estiver em modo de arquivos
    elif exibir_arquivos:
        query_files = f"{query_base} and mimeType!='application/vnd.google-apps.folder'"
        resp_files = drive_service.files().list(
            q=query_files,
            spaces='drive',
            fields="nextPageToken, files(id, name, mimeType)",
            pageSize=ITENS_POR_PAGINA,
            pageToken=None if page_token in ("", "null") else page_token
        ).execute()

        arquivos = resp_files.get("files", [])
        file_token = resp_files.get("nextPageToken", '')
        globals()['DEBUG_MODE'] and print(f"📄 Arquivos carregados: {len(arquivos)}")

    if config.get("pastas_carregadas") and exibir_arquivos:
        # query_files = f"'{parent_id}' in parents and trashed=false and mimeType!='application/vnd.google-apps.folder'"
        # Mapeamento de extensões para MIME types
        # Mapeamento de extensões para MIME types
        extensao_para_mime = {
            ".csv": "text/csv",
            ".txt": "text/plain",
            ".html": "text/html",
            ".json": "application/json",
            ".pdf": "application/pdf",
            ".mp4": "video/mp4",
            ".ipynb": "application/x-ipynb+json"
        }

        # Traduz tipos_permitidos para mimetypes válidos
        tipos_mime = [extensao_para_mime[ext] for ext in tipos_permitidos if ext in extensao_para_mime]

        filtros = []

        # Arquivos com mimeType desejado
        if tipos_mime:
            mime_filtros = " or ".join([f"mimeType='{m}'" for m in tipos_mime])
            filtros.append(f"({mime_filtros})")

        # Arquivos sem extensão (sem ponto no nome)
        filtros.append("not name contains '.'")

        # Junta as condições
        filtro_final = " or ".join(filtros)

        # Query final
        query_files = f"'{parent_id}' in parents and trashed=false and ({filtro_final})"



        resp_files = drive_service.files().list(
            q=query_files,
            spaces='drive',
            fields="nextPageToken, files(id, name, mimeType)",
            pageSize=15,
            pageToken=None
        ).execute()
        arquivos = resp_files.get("files", [])
        file_token = resp_files.get("nextPageToken", '')

    # Começa HTML da interface
    # is_append = "true" if page_token or config.get("pastas_carregadas") else "false"
    is_append = "true" if page_token else "false"

    globals()['DEBUG_MODE'] and print(f"📊 DEBUG: page_token={repr(page_token)} | pastas_carregadas={config.get('pastas_carregadas')} → is_append={is_append}")
    html = f"<div id='drive_ui_container' data-append='{is_append}' style='font-family:sans-serif;border:1px solid #2d2d2d;border-radius:8px;padding:16px;background:#1e1e1e;color:white;'>"

    # Breadcrumbs
    nav_parts = []
    for i in range(len(path_stack) + 1):
        sub = json.dumps(json.dumps(path_stack[:i]))
        name = "Meu Drive" if i == 0 else path_stack[i - 1]['name']
        nav_parts.append(f"""
          <button onclick='
            disableAllDriveButtons(this, "Abrindo");
            google.colab.kernel.invokeFunction("listar_pastas_drive_js", [{json.dumps(caminho)}, "", {sub}, {json.dumps(config_json)}], {{}});
          ' class='breadcrumb-btn' style='background:none;border:none;color:#93c5fd;font-weight:500;cursor:pointer;font-size:14px;padding:4px 6px;border-radius:4px;transition:background 0.2s;' onmouseover='this.style.background=\"#1f2937\"' onmouseout='this.style.background=\"none\"'>{name}</button>
        """)


    html += "<div style='margin-bottom:10px;padding:8px;background:#374151;border-radius:4px;font-size:15px;display:flex;align-items:center;gap:10px;'>"
    if path_stack:
        back = json.dumps(json.dumps(path_stack[:-1]))
        html += f"""
        <button onclick='
          disableAllDriveButtons(this, "Abrindo");
          google.colab.kernel.invokeFunction("listar_pastas_drive_js", [
            {json.dumps(caminho)}, "", {back}, {json.dumps(config_json)}
          ], {{}});
        '
        class='breadcrumb-btn'
        style='background:#4b5563;color:white;padding:6px 10px;border:none;border-radius:6px;cursor:pointer;'>
          ⬅️
        </button>
        """

    html += f"<div style='display:flex;justify-content:space-between;align-items:center;width:100%;'>"
    html += f"<div><strong>📁 </strong>{' / '.join(nav_parts)}</div>"

    # Botão "Salvar aqui"
    if modo == "salvar" and mostrar_botoes_acao:
        html += f"""
        <button onclick='
          disableAllDriveButtons(this, "Salvando");

          // Remove a interface imediatamente
          const div = document.getElementById("input_drive");
          if (div) div.remove();

          // Chama o callback para registrar o caminho escolhido no console do Colab
          google.colab.kernel.invokeFunction("callback_diretorio_escolhido", ["{caminho}"], {{}});

          // Chama o Python para salvar o arquivo
          google.colab.kernel.invokeFunction("upload_para_drive_api", [
            "{caminho}", "{parent_id}"
          ], {{}});
        ' style='background:#10b981;color:white;padding:6px 12px;border:none;border-radius:6px;cursor:pointer;font-size:14px;'>
          💾 Salvar aqui
        </button>
        """


    html += "</div></div>"  # fecha a linha de breadcrumb + botão


    # Estilo
    html += """
    <style>
    .file-btn {
      display: block;
      background: #1e40af;
      color: white;
      padding: 8px 12px;
      border: 2px solid transparent;
      border-radius: 6px;
      cursor: pointer;
      box-shadow: 0 2px 5px rgba(0,0,0,0.3);
      transition: all 0.2s;
    }
    .file-btn:hover {
      background: #1d4ed8;
    }
    .file-btn.selected {
      border-color: white;
    }
    </style>
    """

    # Lista de itens
    html += "<ul style='list-style:none;padding:0;display:flex;flex-wrap:wrap;gap:8px;'>"

    globals()['DEBUG_MODE'] and print("📁 Pastas carregadas:", len(folders))
    globals()['DEBUG_MODE'] and print("📄 Arquivos carregados:", len(arquivos))

    for f in folders:
        nome = f["name"]
        path = json.dumps(json.dumps(path_stack + [f]))
        # path = json.dumps(path_stack + [f])

        html += f"""
        <li>
          <button onclick='
            disableAllDriveButtons(this, "Abrindo");
            google.colab.kernel.invokeFunction("listar_pastas_drive_js", [{json.dumps(caminho)}, "", {path}, {json.dumps(config_json)}], {{}});
          ' class='file-btn' style='background:#2563eb;'>📂 {nome}</button>
        </li>
        """


    for f in arquivos:
      nome = f["name"]
      nome_lower = nome.lower()

      # Considera apenas o "último pedaço" (em caso de URL ou caminho com barras)
      nome_final = nome_lower.split("/")[-1]

      # Detecta se tem extensão real ao final (ex: .csv, .txt, etc)
      match = re.match(r"^.+\.(\w+)$", nome_final)
      extensao = f".{match.group(1)}" if match else ""
      tem_extensao = bool(extensao)

      # Se tem extensão → verificar se é permitida
      if tem_extensao:
          if tipos_permitidos and extensao not in tipos_permitidos:
              globals()['DEBUG_MODE'] and print("⛔ Ignorado:", nome)
              continue
      else:
          # Sem extensão só é permitido se ".txt" estiver permitido
          if ".txt" not in tipos_permitidos:
              globals()['DEBUG_MODE'] and print("⛔ Ignorado (sem extensão não permitida):", nome)
              continue

      globals()['DEBUG_MODE'] and print("✅ Exibido:", nome)
      btn_id = f"btn_{f['id'].replace('-', '')}"
      html += f"""
      <li>
          <button id='{btn_id}' onclick='selectFile("{f['id']}", "{nome}", this)' class='file-btn' style='background:#3b82f6;'>
              📄 {nome}
          </button>
      </li>
      """

    html += "</ul>"

    # Botão de "Carregar mais" ou mensagem final
    if folder_token:
        next_config = config.copy()
        next_config["pastas_carregadas"] = config.get("pastas_carregadas")
        next_token = folder_token
        html += f"""
        <div id="carregar_mais_btn" style='margin-top:20px; display:flex; justify-content:center;'>
          <button onclick='this.disabled=true; this.innerText="⏳ Carregando pastas..."; google.colab.kernel.invokeFunction("listar_pastas_drive_js", [
            {json.dumps(caminho)},
            {"null" if not next_token else f'"{next_token}"'},
            {json.dumps(json.dumps(path_stack))},
            {json.dumps(json.dumps(next_config))}
          ], {{}})'
          style='background:#4b5563;color:white;padding:8px 16px;border:none;border-radius:6px;cursor:pointer;transition:all 0.2s;'>
            🔽 Carregar mais pastas
          </button>
        </div>
        """
    elif not config.get("pastas_carregadas") and exibir_arquivos:
      # Agora que todas as pastas acabaram, começa a carregar arquivos no próximo clique
      next_config = config.copy()
      next_config["pastas_carregadas"] = True
      html += f"""
      <div id="carregar_mais_btn" style='margin-top:20px; display:flex; justify-content:center;'>
        <button onclick='this.disabled=true; this.innerText="⏳ Carregando arquivos..."; google.colab.kernel.invokeFunction("listar_pastas_drive_js", [
          {json.dumps(caminho)},
          null,
          {json.dumps(json.dumps(path_stack))},
          {json.dumps(json.dumps(next_config))}
        ], {{}})'
        style='background:#4b5563;color:white;padding:8px 16px;border:none;border-radius:6px;cursor:pointer;transition:all 0.2s;'>
          🔽 Carregar arquivos
        </button>
      </div>
      """

    else:
        html += "<div id='carregar_mais_btn' style='margin-top:20px; text-align:center; color:#ccc; font-size:14px;'>🚫 Nada mais para carregar.</div>"

    html += "</div>"  # fechamento da div principal
    display(Javascript(f"window.renderFolderUI({json.dumps(html)})"))


def carregar_arquivo_selecionado(file_id, nome_arquivo):
    print(f"📥 Iniciando download do arquivo: {nome_arquivo} (ID: {file_id})")
    from googleapiclient.http import MediaIoBaseDownload
    from io import BytesIO
    auth.authenticate_user()
    drive_service = build('drive', 'v3')

    request = drive_service.files().get_media(fileId=file_id)
    fh = BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()

    print(f"✅ Download completo de {nome_arquivo}")
    html_content = fh.getvalue().decode("utf-8")
    base = nome_arquivo.rsplit('.', 1)[0]
    try:
        print(f"🔁 Convertendo HTML para CSV: {base}")
        converter_html_para_csv(html_content, base)
    except Exception as e:
        print(f"❌ Erro ao converter HTML: {e}")

    display(Javascript("""
      const div = document.getElementById('input_drive');
      if (div) div.remove();
    """))

# def processar_upload_automatico(arquivo_bytes, nome_arquivo):
#     print(f"\n📄 Processando: {nome_arquivo}")
#     html = arquivo_bytes.encode("latin1").decode("utf-8", errors="ignore")
#     base = nome_arquivo.rsplit('.', 1)[0]
#     try:
#         converter_html_para_csv(html, base)
#     except Exception as e:
#         print(f"❌ Erro ao processar {nome_arquivo}: {e}")
def processar_upload_automatico(conteudo, nome_arquivo="agendamentos"):
    if isinstance(conteudo, bytes):
        try:
            texto = conteudo.decode("utf-8")
        except UnicodeDecodeError:
            texto = conteudo.decode("latin1")
    else:
        texto = conteudo  # já é str

    converter_html_para_csv(texto, nome_arquivo)






# Callbacks
output.register_callback('listar_pastas_drive_js', listar_pastas_drive_js)
output.register_callback('criar_pasta', criar_pasta)
output.register_callback('renomear_pasta', renomear_pasta)
output.register_callback('excluir_pasta', excluir_pasta)
output.register_callback('carregar_arquivo_selecionado', carregar_arquivo_selecionado)
output.register_callback('upload_local_auto', processar_upload_automatico)

# output.register_callback('listar_pastas_drive_js', listar_pastas_drive_js)

def upload_para_drive_api(caminho, folder_id):
    from google.colab import auth
    from googleapiclient.discovery import build
    from googleapiclient.http import MediaFileUpload

    auth.authenticate_user()
    drive_service = build('drive', 'v3')

    nome_arquivo = os.path.basename(caminho)
    file_metadata = {'name': nome_arquivo, 'parents': [folder_id]}
    media = MediaFileUpload(caminho, mimetype='text/csv')
    arquivo = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    file_id = arquivo['id']

    file_link = f"https://drive.google.com/file/d/{file_id}/view"
    folder_link = f"https://drive.google.com/drive/u/0/folders/{folder_id}"

    display(HTML(f"""
        <div style='font-family:sans-serif;'>
        <p>📁 Arquivo enviado com sucesso!</p>
        <a href="{file_link}" target="_blank" style="text-decoration: none;">
            <button style='background-color:#3b82f6;color:white;padding:8px 16px;border:none;border-radius:4px;margin-right:10px;cursor:pointer;'>
                🔗 Abrir CSV
            </button>
        </a>
        <a href="{folder_link}" target="_blank" style="text-decoration: none;">
            <button style='background-color:#6366f1;color:white;padding:8px 16px;border:none;border-radius:4px;cursor:pointer;'>
                📂 Abrir Pasta no Drive
            </button>
        </a>
        </div>
    """))

    display(Javascript("""
      const div = document.getElementById('input_drive');
      if (div) div.remove();
    """))


output.register_callback('upload_para_drive_api', upload_para_drive_api)

display(HTML('''
<div id="upload_ui" style="display:flex; gap:10px; margin-top:10px; font-family:sans-serif;">
  <input type="file" id="upload_input_auto" accept=".html,.txt" style="display:none" />
  <button type="button"
          onclick="document.getElementById('upload_input_auto').click();"
          style="background:#3b82f6;color:white;padding:8px 16px;border:none;border-radius:4px;cursor:pointer;">
    📁 Carregar do Computador
  </button>

  <button onclick="document.getElementById('upload_ui')?.remove();
         initDriveUI('', {
           modo: 'abrir',
           tipos_permitidos: ['.html', '.txt'],
           mostrar_arquivos: true,
           mostrar_botao_renomear: false,
           mostrar_botao_excluir: false,
           mostrar_botoes_acao: false
         })"
          style="background:#10b981;color:white;padding:8px 16px;border:none;border-radius:4px;cursor:pointer;">
    ☁️ Carregar do Drive
  </button>
</div>
<div id="input_drive"></div>
'''))

display(Javascript("""
(() => {
  const input = document.getElementById('upload_input_auto');
  if (!input) return;

  input.addEventListener('change', function() {
    const file = this.files[0];
    if (!file) return;

    const reader = new FileReader();
    reader.onload = function(e) {
      // Chama função Python para logar no console do Colab
      google.colab.kernel.invokeFunction(
        'log_selecao_local',
        [file.name],
        {}
      );

      // Remove UI inicial
      document.getElementById('upload_ui')?.remove();

      // Continua o processamento
      google.colab.kernel.invokeFunction(
        'upload_local_auto',
        [e.target.result, file.name],
        {}
      );
    };


    reader.readAsText(file);
  });
})();
"""))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>